# CrewAI Demo

CrewAI orchestrates role-based multi-agent teams that mimic real-world human crew dynamics.

## Required Environment Variables

```
OPENAI_API_KEY
SERPER_API_KEY  # Optional, for web search
```

In [ ]:
# If running standalone without the project setup:
# pip install crewai crewai-tools openai python-dotenv

In [1]:
import os
import yaml
from pathlib import Path
from dotenv import load_dotenv
from typing import Any, Type, Optional
from pydantic import BaseModel, Field

# Load environment variables
load_dotenv(override=True)

# Import CrewAI components
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import SerperDevTool
from crewai.tools import BaseTool

/Users/uzyn/Projects/NUS/agentic-ai-course-private/3-module/.venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)


## What We're Building

A **Content Marketing Team** — three agents (researcher, writer, editor) collaborating to produce a blog post about AI-powered content marketing.

In [2]:
llm = LLM(model="gpt-4o", temperature=0.7)

web_search_tool = None
try:
    if os.getenv('SERPER_API_KEY'):
        web_search_tool = SerperDevTool()
        print("Web Search Tool (SerperDev) initialized")
    else:
        print("No SERPER_API_KEY found — agents will use LLM knowledge only")
except Exception as e:
    print(f"Web Search Tool error: {e}")

Web Search Tool (SerperDev) initialized


## Agent Definition

Agents are role-based team members with a **role**, **goal**, **backstory**, and **tools**.

In [6]:
agents_yaml_content = """
content_researcher:
  role: Senior Content Researcher
  goal: Research and gather comprehensive information on given topics to support content creation
  backstory: Experienced researcher with 10+ years in digital marketing, skilled at finding trends and actionable insights.
  tools:
  - web_search
  verbose: true
  allow_delegation: false

content_writer:
  role: Senior Content Writer
  goal: Create engaging, high-quality content based on research and brand guidelines
  backstory: Seasoned content writer who transforms research into compelling marketing narratives.
  tools: []
  verbose: true
  allow_delegation: false

content_editor:
  role: Senior Content Editor
  goal: Review, refine, and ensure content quality meets brand standards and objectives
  backstory: Experienced editor with a keen eye for detail, brand consistency, and audience engagement.
  tools: []
  verbose: true
  allow_delegation: true
"""

agents_config = yaml.safe_load(agents_yaml_content.strip())

agents_config

{'content_researcher': {'role': 'Senior Content Researcher',
  'goal': 'Research and gather comprehensive information on given topics to support content creation',
  'backstory': 'Experienced researcher with 10+ years in digital marketing, skilled at finding trends and actionable insights.',
  'tools': ['web_search'],
  'verbose': True,
  'allow_delegation': False},
 'content_writer': {'role': 'Senior Content Writer',
  'goal': 'Create engaging, high-quality content based on research and brand guidelines',
  'backstory': 'Seasoned content writer who transforms research into compelling marketing narratives.',
  'tools': [],
  'verbose': True,
  'allow_delegation': False},
 'content_editor': {'role': 'Senior Content Editor',
  'goal': 'Review, refine, and ensure content quality meets brand standards and objectives',
  'backstory': 'Experienced editor with a keen eye for detail, brand consistency, and audience engagement.',
  'tools': [],
  'verbose': True,
  'allow_delegation': True}}

In [ ]:
tools_map = {'web_search': web_search_tool}

agents = {}
for agent_name, config in agents_config.items():
    agent_tools = [tools_map[t] for t in config.get('tools', []) if t in tools_map and tools_map[t]]

    backstory = config['backstory']
    if config.get('allow_delegation', False):
        backstory += "\n\nIMPORTANT: When delegating tasks, provide parameters as plain strings, not as dictionary objects."

    agents[agent_name] = Agent(
        role=config['role'],
        goal=config['goal'],
        backstory=backstory,
        tools=agent_tools,
        verbose=config.get('verbose', True),
        allow_delegation=config.get('allow_delegation', False),
        llm=llm,
        max_iter=5
    )

researcher = agents['content_researcher']
writer = agents['content_writer']
editor = agents['content_editor']

for name, agent in agents.items():
    print(f"{agent.role}: {len(agent.tools)} tool(s), delegation={'yes' if agent.allow_delegation else 'no'}")

{'content_researcher': Agent(role=Senior Content Researcher, goal=Research and gather comprehensive information on given topics to support content creation, backstory=Experienced researcher with 10+ years in digital marketing, skilled at finding trends and actionable insights.),
 'content_writer': Agent(role=Senior Content Writer, goal=Create engaging, high-quality content based on research and brand guidelines, backstory=Seasoned content writer who transforms research into compelling marketing narratives.),
 'content_editor': Agent(role=Senior Content Editor, goal=Review, refine, and ensure content quality meets brand standards and objectives, backstory=Experienced editor with a keen eye for detail, brand consistency, and audience engagement.
 
 IMPORTANT: When delegating tasks, provide parameters as plain strings, not as dictionary objects.)}

## Task Definition

Tasks are structured work items with a **description**, **expected output**, **assigned agent**, and optional **context** (dependencies on other tasks).

In [ ]:
tasks_yaml_content = """
research_task:
  description: Research the latest trends and best practices in AI-powered content marketing for 2024. Focus on practical applications, case studies, and emerging technologies.
  agent: content_researcher
  expected_output: A comprehensive research report with key trends, statistics, and actionable insights.
  tools_used:
  - web_search

writing_task:
  description: Create an engaging blog post about AI-powered content marketing based on the research findings. Target audience is marketing professionals.
  agent: content_writer
  expected_output: A well-structured 1200-1500 word blog post with introduction, key sections, examples, and conclusion.
  context:
  - research_task
  tools_used: []

editing_task:
  description: Review and refine the blog post for clarity, engagement, brand consistency, and SEO optimization.
  agent: content_editor
  expected_output: A polished, publication-ready blog post with improved flow, grammar, and SEO.
  context:
  - research_task
  - writing_task
  tools_used: []
"""

tasks_config = yaml.safe_load(tasks_yaml_content.strip())

for task_name, config in tasks_config.items():
    deps = config.get('context', ['None'])
    print(f"{task_name}: agent={config['agent']}, depends_on={', '.join(deps)}")

In [ ]:
tasks = {}

for task_name, config in tasks_config.items():
    context = [tasks[dep] for dep in config.get('context', []) if dep in tasks]

    tasks[task_name] = Task(
        description=config['description'],
        agent=agents[config['agent']],
        expected_output=config['expected_output'],
        context=context if context else None
    )

research_task = tasks['research_task']
writing_task = tasks['writing_task']
editing_task = tasks['editing_task']

for i, (name, task) in enumerate(tasks.items(), 1):
    deps = [n for n, t in tasks.items() if task.context and t in (task.context if isinstance(task.context, list) else [task.context])]
    print(f"{i}. {name} -> {task.agent.role} (depends: {', '.join(deps) or 'none'})")

## Crew Orchestration

The **Crew** orchestrates agents and tasks. Process types: **sequential**, **hierarchical**, and **consensual**.

In [ ]:
content_crew = Crew(
    agents=[researcher, writer, editor],
    tasks=[research_task, writing_task, editing_task],
    process=Process.sequential,
    verbose=True,
    memory=False,
    max_iter=10
)

print(f"Crew: {len(content_crew.agents)} agents, {len(content_crew.tasks)} tasks, process={content_crew.process.value}")

## Execute Crew

In [ ]:
result = content_crew.kickoff()

print("\nFinal Deliverable:")
print(result)

## Summary

CrewAI provides role-based multi-agent orchestration with:
- **Agents**: Role-based team members with goals, backstories, and tools
- **Tasks**: Structured work items with dependencies and expected outputs
- **Crew**: Orchestrates agents through sequential, hierarchical, or consensual processes
- **YAML config**: Scalable agent/task definitions without code changes

### Resources
- [CrewAI Documentation](https://docs.crewai.com/)
- [GitHub: crewAI-Inc/crewAI](https://github.com/crewAI-Inc/crewAI)